## Pandas Time Series - Practice



### Parking garage Dataset
* The given data set (transations.csv) consists of more than three years of parking transaction data.
* (from Jan 1, 2013 to Jan 31, 2016). 

* There are two columns: entry time and exit time. 
* Each row represents one transaction: For example: 
    * entry time, exit time 
    * 2015-01-01 01:32:25, 2015-01-01 03:02:52 
    * 2015-01-01 01:58:11, 2015-01-01 06:39:03 
    * ... 

### Requirements
* Part1: Data cleaning.
* Show a graph of #parking/month from jan1, 2013 to jan31 2016.
* Show a graph of #parking /day from jan1 2016 to jan31 2016.
* Show a graph of #parking/hour on jan1 2016.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#================ cleaning ===========================
# read in transaction data and parse dates
data = pd.read_csv(r"./DATA/transactions.csv", parse_dates = ['entry_time','exit_time'])
data.head()

In [ ]:
# replace all second value to be 0
entry_list = data['entry_time']
entry_list = [t.replace(second=0) for t in entry_list]
entry_list

In [ ]:
exit_list = data['exit_time']
exit_list = [t.replace(second=0) for t in exit_list]
exit_list

In [ ]:
# entry_time/exit_time dataframe
data = pd.DataFrame({'entry_time':entry_list, 'exit_time':exit_list})
data.head()

In [ ]:
# remove data when it has entry time>exit time
data[data.entry_time>data.exit_time]

In [ ]:
# remove data when it has entry time>exit time
data = data[data.entry_time<data.exit_time]

In [ ]:
# entry/exit 1/-1 list
data_entry = pd.DataFrame({'time' :data['entry_time'], 'N':[1] * len(data)})
data_exit = pd.DataFrame({'time' :data['exit_time'], 'N':[-1]*len(data)})


In [ ]:
data_entry.head()

In [ ]:
data_exit.head()

In [ ]:
# merge entry and exit 1/-1 list and sort it
data_merge = data_entry.append(data_exit)
data_merge = data_merge.sort_values(by = 'time', ascending = True)
data_merge

In [ ]:
# cumulative sum of the sorted dataframe
data_merge['cumsum'] = np.cumsum(data_merge['N'])
data_merge

In [ ]:
# reset index as time
data_merge1 = pd.DataFrame({'cumsum':data_merge['cumsum'], 'time':data_merge['time']})
data_merge2 = data_merge1.set_index('time')
data_merge2

In [ ]:
# pick the last value if there're duplicate time
data_merge3 = data_merge2[~data_merge2.index.duplicated(keep='last')]
data_merge3

In [ ]:
# upsample the frame and fill in NA minutes with the previous existed value
resample = data_merge3.resample('1T',fill_method = 'pad')
resample

In [ ]:
# remove 14 records before 2013
resample1 = resample['2013-01-01':'2016-01-31']
resample1

In [ ]:
##================= visualization =============
#Graph 1
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10


r1 = resample1['1/1/2013':'1/31/2016']
r1=r1.resample('1M', how='mean')

r1.plot(kind='bar',legend=None)
plt.xlabel('Month',size=20)
plt.ylabel('Number of Cars',size=20)
plt.title('Average Density of Cars per Month',size=30)
plt.xticks(rotation=70)



In [ ]:
#Graph 2
r2 = resample1['1/1/2016':'1/31/2016']
r2=r2.resample('1D', how='mean')
r2.plot(kind='bar',legend=None)


plt.xlabel('Day',size=20)
plt.ylabel('Number of Cars',size=20)
plt.title('Average Density of Cars per Day',size=30)
plt.xticks(rotation=70)



In [ ]:
#Graph 3
from pylab import rcParams
#%matplotlib inline
rcParams['figure.figsize'] = 20, 10
r3 = resample1['1/1/2013']
r3=r3.resample('1H', how='mean')
r3.plot(kind='bar',legend=None)

plt.xlabel('Hour of Day',size=20)
plt.ylabel('Number of Cars', size=20)
plt.title('Average Density of Cars per Hour',size=30)
plt.xticks(rotation=70)



## prediction

* pip install holtwinters
* https://grisha.org/blog/2016/01/29/triple-exponential-smoothing-forecasting/

In [ ]:
## ================= prediction =================
#import pycast
import holtwinters
#import datetime
#import matplotlib.dates as mdates
# additive model prediction hourly, forecasting 696 hours
r4 = resample1.resample('1H', how = 'mean')
result = holtwinters.additive(x = r4['cumsum'].tolist(), m = 24, fc = 696)
forecast = pd.DataFrame(result[0])
print(forecast.plot(kind = 'bar'))